In [1]:
# For files e data manipulations
from imports_ import *

## Neural Network

In [2]:
pnn = PreNeuralNetwork(train='C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/train', test='C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/test' ,
                    valid='C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/validation')

In [3]:
dir_train = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/train'
dir_test  = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/test'
dir_valid = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/validation'

batch_size = 4  
image_size = (224, 224)

random.seed(123)

In [4]:
train_datagen= ImageDataGenerator(rescale=1.0/255)
val_datagen  = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator=train_datagen.flow_from_directory(dir_train,
                                        target_size=image_size,
                                        color_mode='rgb',  # Ensure RGB color mode
                                        class_mode='categorical',  # Adjust class_mode as needed
                                        batch_size=batch_size,
                                        subset='training',  # Specify 'training' for the training set
                                    )

val_generator=val_datagen.flow_from_directory( dir_valid,
                                        target_size=image_size,
                                        color_mode='rgb',
                                        class_mode='sparse',batch_size=batch_size)

test_generator=test_datagen.flow_from_directory(dir_test,
                                        target_size=image_size,
                                        color_mode='rgb',
                                        class_mode='sparse',batch_size=batch_size)


Found 77877 images belonging to 9 classes.
Found 4328 images belonging to 9 classes.
Found 4326 images belonging to 9 classes.


In [5]:
train_generator

In [6]:
n_classes = train_generator.num_classes
print("Number of classes:", n_classes)

Number of classes: 9


In [7]:

class ResNet101Model:
    def __init__(self, num_classes, image_shape):
        self.num_classes = num_classes
        self.image_shape = image_shape
        self.model = self.build_model()
        self.metrics_df = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss', 'train_accuracy', 'val_accuracy'])
    
    def build_model(self):
        base_model = ResNet101(weights='imagenet', include_top=False, input_shape=self.image_shape)
        x = GlobalAveragePooling2D()(base_model.output)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        for layer in base_model.layers:
            layer.trainable = False
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model
    
    def train(self, train_data, val_data, epochs, batch_size, early_stop_patience=5):
        early_stop_callback = EarlyStopping(monitor='val_loss', patience=early_stop_patience, restore_best_weights=True)
        
        history = self.model.fit(
            train_data,
            validation_data=val_data,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stop_callback]
        )
        
        self.update_metrics(history)
    
    def update_metrics(self, history):
        for epoch, train_loss, val_loss, train_accuracy, val_accuracy in zip(
                range(len(history.history['loss'])),
                history.history['loss'],
                history.history['val_loss'],
                history.history['accuracy'],
                history.history['val_accuracy']
        ):
            self.metrics_df = self.metrics_df.append({
                'epoch': epoch + 1,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_accuracy': train_accuracy,
                'val_accuracy': val_accuracy
            }, ignore_index=True)
        return self.metrics_df
    def evaluate(self, test_data):
        return self.model.evaluate(test_data)


In [8]:

if tf.test.is_gpu_available():
    print("GPU is available and accessible.")
else:
    print("GPU is not available.")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is not available.


In [9]:
# Assuming you have a GPU with index 0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Specify GPU device
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [10]:
5641jyhgvuy

SyntaxError: invalid imaginary literal (3556377177.py, line 1)

In [11]:
# Create an instance of ResNet101Model
num_classes = n_classes  # Adjust the number of classes as needed
image_shape = (224,224,3)  # Adjust image dimensions as needed
model = ResNet101Model(num_classes, image_shape)

# Train the model
model.train(train_generator, val_generator, epochs=10, batch_size=batch_size, early_stop_patience=5)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)

# Access and analyze training and validation metrics
metrics_df = model.metrics_df
print(metrics_df)

Epoch 1/10
  38/2434 [..............................] - ETA: 1:34:28 - loss: 1.2936 - accuracy: 0.6678

In [ ]:
# # vgg16 = keras.applications.vgg16
# vgg = vgg16.VGG16(input_shape = (img_width, img_height, 3), 
#                        include_top = False, 
#                        weights = 'imagenet')

# # vgg.summary()
# # Layer of transfer learning not trainable
# # vgg.trainable = False
# for layer in vgg.layers:
#   layer.trainable = False
  
# output = vgg.output
# # three hidden layers
# # output = keras.layers.GlobalAveragePooling2D()(output)
# output = keras.layers.Flatten()(output)
# output = keras.layers.Dense(512, activation='relu')(output)
# output = keras.layers.Dropout(0.75)(output) 
# output = BatchNormalization()(output)
# output = keras.layers.Dropout(0.5)(output) 
# # output = keras.layers.Dense(22, activation='relu')(output)
# # final softmax layer

# predictions = keras.layers.Dense(len(bodyparts), activation='softmax')(output)

# # creating the full model:
# full_model = keras.models.Model(inputs=vgg.input, outputs=predictions)
# full_model.summary()

In [ ]:
full_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
! mkdir model_NIG-CHEST

In [ ]:
# Checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'model_NIG-CHEST/model_vgg_{qntd_for_model}_1.h5', 
                                                verbose = 1, 
                                                save_best = True, 
                                                save_weights_only = True)

# Early stop
early_stop = tf.keras.callbacks.EarlyStopping(patience = 4) 

In [ ]:
history = full_model.fit(train_ds, 
                        epochs=50,
                        validation_data=val_ds,
                        workers=5,
                        callbacks = [checkpoint, early_stop])

In [ ]:
plot_history(history)